In [1]:
# For these lessons we will need NumPy, pandas, matplotlib and seaborn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# and of course the actual regression (machine learning) module
from sklearn import linear_model
# from sklearn.linear_model import LinearRegression

In [49]:
data = pd.read_csv("1.02.+Multiple+linear+regression.csv")
data.head(10)

,SAT,"Rand 1,2,3",GPA
0,1714,1,2.40
1,1664,3,2.52
2,1760,3,2.54
3,1685,3,2.74
4,1693,2,2.83
5,1670,1,2.91
6,1764,2,3.00
7,1764,1,3.00
8,1792,2,3.01
9,1850,3,3.01


In [3]:
x = data[['SAT','Rand 1,2,3']]
y = data['GPA']

In [4]:
# Since the p-values are obtained through certain statistics, we need the 'stat' module from scipy.stats
import scipy.stats as stat

# Since we are using an object oriented language such as Python, we can simply define our own 
# LinearRegression class (the same one from sklearn)
# By typing the code below we will ovewrite a part of the class with one that includes p-values
# Here's the full source code of the ORIGINAL class: https://github.com/scikit-learn/scikit-learn/blob/7b136e9/sklearn/linear_model/base.py#L362


class LinearRegression(linear_model.LinearRegression):
    """
    LinearRegression class after sklearn's, but calculate t-statistics
    and p-values for model coefficients (betas).
    Additional attributes available after .fit()
    are `t` and `p` which are of the shape (y.shape[1], X.shape[1])
    which is (n_features, n_coefs)
    This class sets the intercept to 0 by default, since usually we include it
    in X.
    """
    
    # nothing changes in __init__
    def __init__(self, fit_intercept=True, normalize=False, copy_X=True,
                 n_jobs=1, positive= False):
        self.fit_intercept = fit_intercept
        self.normalize = normalize
        self.copy_X = copy_X
        self.n_jobs = n_jobs
        #force to 
        self.positive = positive
    
    def fit(self, X, y, n_jobs=1):
        self = super(LinearRegression, self).fit(X, y, n_jobs)
        
        # Calculate SSE (sum of squared errors)
        # and SE (standard error)
        sse = np.sum((self.predict(X) - y) ** 2, axis=0) / float(X.shape[0] - X.shape[1])
        se = np.array([np.sqrt(np.diagonal(sse * np.linalg.inv(np.dot(X.T, X))))])

        # compute the t-statistic for each feature
        self.t = self.coef_ / se
        # find the p-value for each feature
        self.p = np.squeeze(2 * (1 - stat.t.cdf(np.abs(self.t), y.shape[0] - X.shape[1])))
        return self

In [5]:
reg = LinearRegression()
reg.fit(x,y)

LinearRegression()

In [6]:
coef = reg.coef_
intercept = reg.intercept_
print(coef,intercept)

[ 0.00165354 -0.00826982] 0.29603261264909575


In [7]:
rSquared = reg.score(x,y)
rSquared

0.4066811952814282

In [57]:
new_data =pd.DataFrame({'SAT':[1740],'Rand 1,2,3':[3]})
reg.predict(new_data)

array([302.26532127])

In [9]:
#function to caculate adjusted R squared
def adjustedR2(n,p,r2):
    return 1-(1-r2)*(n-1)/(n-p-1)

In [10]:
observationsSize = x.shape[0]
featuresOrInputSize = x.shape[1]

adjustedR2(observationsSize,featuresOrInputSize,rSquared)

0.39203134825134

In [11]:
# from sklearn.feature_selection import f_regression
# fRegression = f_regression(x,y)
# pValues = fRegression[1]
# pValues

In [12]:
# # Above values in array cant be read, So make them understandable we can round them
# # if you find any p-value of feature/input more than 0.5 (value>0.5) just remove that feature/input.
# pValues.round(3)

In [13]:
# T_statistics
reg.t

array([[51.45496302, -0.3102333 ]])

In [14]:
reg_summary = pd.DataFrame({'Featues':x.columns.values})
# P-values without using f_regression
reg_summary['P Values'] = reg.p.round(3)
reg_summary['Coefficient'] = reg.coef_
reg_summary

,Featues,P Values,Coefficient
0,SAT,0.000,0.001654
1,"Rand 1,2,3",0.757,-0.008270


In [15]:
reg.intercept_

0.29603261264909575

In [16]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaler.fit(x)

StandardScaler()

In [17]:
x_scaler = scaler.transform(x)
x_scaler

array([[-1.26338288, -1.24637147],
       [-1.74458431,  1.10632974],
       [-0.82067757,  1.10632974],
       [-1.54247971,  1.10632974],
       [-1.46548748, -0.07002087],
       [-1.68684014, -1.24637147],
       [-0.78218146, -0.07002087],
       [-0.78218146, -1.24637147],
       [-0.51270866, -0.07002087],
       [ 0.04548499,  1.10632974],
       [-1.06127829,  1.10632974],
       [-0.67631715, -0.07002087],
       [-1.06127829, -1.24637147],
       [-1.28263094,  1.10632974],
       [-0.6955652 , -0.07002087],
       [ 0.25721362, -0.07002087],
       [-0.86879772,  1.10632974],
       [-1.64834403, -0.07002087],
       [-0.03150724,  1.10632974],
       [-0.57045283,  1.10632974],
       [-0.81105355,  1.10632974],
       [-1.18639066,  1.10632974],
       [-1.75420834,  1.10632974],
       [-1.52323165, -1.24637147],
       [ 1.23886453, -1.24637147],
       [-0.18549169, -1.24637147],
       [-0.5608288 , -1.24637147],
       [-0.23361183,  1.10632974],
       [ 1.68156984,

In [18]:
reg = LinearRegression()
reg.fit(pd.DataFrame(data=x_scaler,columns=['SAT','Rand 1,2,3']),y)

LinearRegression()

In [19]:
type(x.columns.values)

numpy.ndarray

In [20]:
summaryData = {'Features':np.append('Bias',x.columns.values), 'Weights': [reg.intercept_,reg.coef_[0],reg.coef_[1]]} 
scalerSummary = pd.DataFrame(summaryData)
scalerSummary

,Features,Weights
0,Bias,3.330238
1,SAT,0.171814
2,"Rand 1,2,3",-0.007030


In [21]:
new_data = pd.DataFrame(data=[[1700,2],[1800,1]],columns=['SAT','Rand 1,2,3'])
new_data

,SAT,"Rand 1,2,3"
0,1700,2
1,1800,1


In [22]:
reg.predict(new_data)

array([295.39979563, 312.58821497])

In [23]:
new_scaler_data = scaler.transform(new_data)
new_scaler_data

array([[-1.39811928, -0.07002087],
       [-0.43571643, -1.24637147]])

In [24]:
reg.predict(new_scaler_data)

/opt/anaconda3/lib/python3.11/site-packages/sklearn/base.py:439: UserWarning: X does not have valid feature names, but LinearRegression was fitted with feature names
  warnings.warn(


array([3.09051403, 3.26413803])